<a href="https://colab.research.google.com/github/carloscesar182/ai_advanced_course/blob/main/Notebooks/MLAdvTechniques/EngAtributos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
dataset = pd.read_csv('credit_simple.csv', sep=';')
dataset.shape

In [ ]:
dataset.head()

In [ ]:
# defining as variaveis X e y
y = dataset['CLASSE']
X = dataset.iloc[:,:-1]

In [ ]:
# ver objetos faltantes em X
X.isnull().sum()

In [ ]:
# calcular a mediana do saldo atual
mediana = X['SALDO_ATUAL'].median()
mediana

In [ ]:
# atualizar o saldo atual com a mediana encontrada
X['SALDO_ATUAL'].fillna(mediana, inplace=True)

# verificar se ficou algo nulo
X.isnull().sum()

In [ ]:
# agrupar o estado civil pra ver a moda
agrupado = X.groupby(['ESTADOCIVIL']).size()
agrupado

In [ ]:
# aplicar a moda no estado civil
X['ESTADOCIVIL'].fillna('masculino solteiro', inplace=True)
X.isnull().sum()

In [ ]:
# verificar outliers (valores fora do padrão)
# usar tecnica simples de desvio padrão, mas tem outras já prontas e mais completas
desv = X['SALDO_ATUAL'].std()
desv

685936688.9820064

In [ ]:
# checar se temos algum lugar com 2x o desvio padrão
X.loc[X['SALDO_ATUAL'] >= 2*desv, 'SALDO_ATUAL']

In [ ]:
# calcular mediana do saldo atual
mediana = X['SALDO_ATUAL'].median()
mediana

2323.0

In [ ]:
# atualizar as 2 linhas que tiveram 2x desv padrão pra mediana
X.loc[X['SALDO_ATUAL'] >= 2*desv, 'SALDO_ATUAL'] = mediana
X.loc[X['SALDO_ATUAL'] >= 2*desv]

In [ ]:
# data binding = remover itens duplicados por digital errada ou de pouca relevancia
agrupado = X.groupby(['PROPOSITO']).size()
agrupado

In [ ]:
# transformação
# eletrodomesticos e qualificação estão pouco representados, então vamos transofrmar em 'outros'
X.loc[X['PROPOSITO'] == 'Eletrodomésticos', 'PROPOSITO'] = 'outros'
X.loc[X['PROPOSITO'] == 'qualificação', 'PROPOSITO'] = 'outros'

# verificar o resultado do agrupamento
agrupado = X.groupby(['PROPOSITO']).size()
agrupado

In [ ]:
# extração de características
X['DATA']

In [ ]:
# colocar no formato padrão usando pandas
X['DATA'] = pd.to_datetime(X['DATA'], format='%d/%m/%Y')
X['DATA']

In [ ]:
# criar 3 atributos para a data: ano, mes e dia
X['ANO'] = X['DATA'].dt.year
X['MES'] = X['DATA'].dt.month
X['DIASEMANA'] = X['DATA'].dt.day_name()

In [ ]:
X['DIASEMANA']

In [ ]:
# label encoder pra informações categóricas ele atribui um valor numérico
# alguns modelos são capazes de converter sozinhos mas a maioria não
# é muito usando em redes neurais
X['ESTADOCIVIL'].unique()
X['PROPOSITO'].unique()
X['DIASEMANA'].unique()

array(['Tuesday', 'Wednesday', 'Thursday', 'Saturday', 'Sunday', 'Monday',
       'Friday'], dtype=object)

In [ ]:
# criar o label encoder
labelencoder1 = LabelEncoder()
X['ESTADOCIVIL'] = labelencoder1.fit_transform(X['ESTADOCIVIL'])
X['PROPOSITO'] = labelencoder1.fit_transform(X['PROPOSITO'])
X['DIASEMANA'] = labelencoder1.fit_transform(X['DIASEMANA'])
X.head()

In [ ]:
# ver a cardinalidade na coluna OUTROSPLANOSPGTO
outros = X['OUTROSPLANOSPGTO'].unique()
outros

In [ ]:
# aplicar o one-hot encoding na coluna OUTROSPLANOSPGTO
z = pd.get_dummies(X['OUTROSPLANOSPGTO'], prefix='OUTROS')
z

In [ ]:
# analizar X
X

In [ ]:
# padronizar com standard scaler os atributos numericos
sc = StandardScaler()
m = sc.fit_transform(X.iloc[:,:3])
m

In [ ]:
# juntar z e m no dataframe original
X = pd.concat([X, z,pd.DataFrame(m,columns=['SALDO_ATUAL_N','RESIDENCIA_DESDE_N','IDADE_N'])], axis=1)
X.head()

In [ ]:
# dropar colunas desnecessarias
X.drop(columns=['SALDO_ATUAL','RESIDENCIADESDE','IDADE','OUTROSPLANOSPGTO','DATA','OUTROS_banco'], inplace=True)
X